In [ ]:
!pip install kaggle
!mkdir .kaggle
import json
token = {"username":"xinkaichen97","key":"a31b1965f229c3f21b0282be2d9cdd3f"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)
!chmod 600 /content/.kaggle/kaggle.json
!cp /content/.kaggle/kaggle.json /root/.kaggle/
!kaggle config path -p /content

In [ ]:
!pip install ipdb
import os, zipfile, tarfile, ipdb
os.environ['KAGGLE_USERNAME'] = "xinkaichen97" 
os.environ['KAGGLE_KEY'] = "a31b1965f229c3f21b0282be2d9cdd3f"

In [ ]:
!kaggle datasets download -d cmu11785/20fall-hw2p2

In [ ]:
!unzip 20fall-hw2p2.zip

In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision.transforms import ToTensor
from torchvision.datasets import ImageFolder
import numpy as np
from sklearn.metrics import roc_auc_score
from collections import OrderedDict
from torch import Tensor
import os

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc. 
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

In [ ]:
# data augmentation
aug_transforms_1 = torchvision.transforms.Compose([
    torchvision.transforms.RandomHorizontalFlip(p=1),
    torchvision.transforms.ToTensor(),
])
aug_transforms_2 = torchvision.transforms.Compose([
    torchvision.transforms.ColorJitter(hue=.05, saturation=.05),
    torchvision.transforms.RandomHorizontalFlip(p=0.5),
    torchvision.transforms.RandomRotation(20, resample=Image.BILINEAR),
    torchvision.transforms.ToTensor(),
])

In [ ]:
transforms = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
])

In [ ]:
train_original = ImageFolder('classification_data/train_data', transform=transforms)
train_augmented_1 = ImageFolder('classification_data/train_data', transform=aug_transforms_1)
train_augmented_2 = ImageFolder('classification_data/train_data', transform=aug_transforms_2)
train_loader = torch.utils.data.ConcatDataset([train_original, train_augmented_1, train_augmented_2])
print(len(train_loader))

In [ ]:
# train_loader = ImageFolder('classification_data/train_data', transform=ToTensor())
val_loader = ImageFolder('classification_data/val_data', transform=ToTensor())
test_loader = ImageFolder('classification_data/test_data', transform=ToTensor())

In [ ]:
print(len(train_loader))
print(train_loader[580637][0].shape, train_loader[580637][1])

In [ ]:
print(len(val_loader))
print(len(test_loader))

In [ ]:
batch_train = DataLoader(train_loader, batch_size=200, shuffle=True, num_workers=4)
batch_val = DataLoader(val_loader, batch_size=200, shuffle=False, num_workers=4)
batch_test = DataLoader(test_loader, batch_size=200, shuffle=False, num_workers=4)

In [ ]:
# baseline
class Net(nn.Module):
  def __init__(self):
        super(Net, self).__init__()
        self.layer1 = nn.Sequential(nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(64), nn.ReLU(), nn.Dropout(0.3),)
        self.layer2 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(64), nn.ReLU(), nn.MaxPool2d(kernel_size=2, stride=2),)
        self.layer3 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(64), nn.ReLU(), nn.MaxPool2d(kernel_size=2, stride=2),)
        self.layer4 = nn.Sequential(nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(128), nn.ReLU(), nn.Dropout(0.3),)
        self.layer5 = nn.Sequential(nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(128), nn.ReLU(), nn.MaxPool2d(kernel_size=2, stride=2),)
        self.layer6 = nn.Sequential(nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(256), nn.ReLU(), nn.Dropout(0.3),)
        self.layer7 = nn.Sequential(nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(256), nn.ReLU(), nn.MaxPool2d(kernel_size=2, stride=2),)
        self.layer8 = nn.Sequential(nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(256), nn.ReLU(), nn.MaxPool2d(kernel_size=2, stride=2),)
        self.layer9 = nn.Sequential(nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(256), nn.ReLU(), nn.MaxPool2d(kernel_size=2, stride=2),)
        # self.layer10 = nn.Sequential(nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(256), nn.ReLU(),)
        # self.layer11 = nn.Sequential(nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(256), nn.ReLU(),)
        # self.layer12 = nn.Sequential(nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(256), nn.ReLU(), nn.AvgPool2d(kernel_size=2, stride=2),)
        self.layer13 = nn.Flatten(start_dim=1)
        self.fc = nn.Linear(256, 4000, bias=False)
        # self.residual = nn.Identity()
  
  def forward(self, x):
        y = self.layer1(x)
        y = self.layer2(y)
        y = self.layer3(y)
        y = self.layer4(y)
        y = self.layer5(y)
        y = self.layer6(y)
        y = self.layer7(y)
        y = self.layer8(y)
        y = self.layer9(y)
        # y = self.layer10(y)
        # y = self.layer11(y)
        # y = self.layer12(y)
        y = self.layer13(y)
        classification = self.fc(y)
        return y, classification

In [ ]:
# similar to VGG-19 
class Net(nn.Module):
  def __init__(self):
        super(Net, self).__init__()
        self.layer1 = nn.Sequential(nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(64), nn.ReLU(),)
        self.layer2 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(64), nn.ReLU(),)
        self.layer3 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(64), nn.ReLU(), nn.MaxPool2d(kernel_size=2, stride=2),)
        self.layer4 = nn.Sequential(nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(128), nn.ReLU(),)
        self.layer5 = nn.Sequential(nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(128), nn.ReLU(),)
        self.layer6 = nn.Sequential(nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(128), nn.ReLU(), nn.MaxPool2d(kernel_size=2, stride=2),)
        self.layer7 = nn.Sequential(nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(256), nn.ReLU(),)
        self.layer8 = nn.Sequential(nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(256), nn.ReLU(),)
        self.layer9 = nn.Sequential(nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(256), nn.ReLU(), nn.MaxPool2d(kernel_size=2, stride=2),)
        self.layer10 = nn.Sequential(nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(512), nn.ReLU(),)
        self.layer11 = nn.Sequential(nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(512), nn.ReLU(),)
        self.layer12 = nn.Sequential(nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(512), nn.ReLU(), nn.MaxPool2d(kernel_size=2, stride=2),)
        self.layer13 = nn.Sequential(nn.Conv2d(512, 1024, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(1024), nn.ReLU(),)
        self.layer14 = nn.Sequential(nn.Conv2d(1024, 1024, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(1024), nn.ReLU(),)
        self.layer15 = nn.Sequential(nn.Conv2d(1024, 1024, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(1024), nn.ReLU(),)
        self.layer16 = nn.Sequential(nn.Conv2d(1024, 1024, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(1024), nn.ReLU(), nn.MaxPool2d(kernel_size=2, stride=2),)
        self.layer17 = nn.Sequential(nn.Conv2d(1024, 2048, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(2048), nn.ReLU(), )
        self.layer18= nn.Sequential(nn.Conv2d(2048, 2048, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(2048), nn.ReLU(),)
        self.layer19 = nn.Sequential(nn.Conv2d(2048, 2048, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(2048), nn.ReLU(),)
        self.layer20 = nn.Sequential(nn.Conv2d(2048, 1024, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(1024), nn.ReLU(),)
        self.layer21 = nn.Sequential(nn.Conv2d(1024, 512, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(512), nn.ReLU(), nn.AvgPool2d(kernel_size=2, stride=2),)
        self.layer22 = nn.Flatten(1)
        self.layer23 = nn.Linear(512, 4000)
        # self.linear_closs = nn.Linear(2048, 2048, bias=False)
        # self.relu_closs = nn.ReLU(inplace=True)
  
  def forward(self, x):
        y = self.layer1(x)
        y = self.layer2(y)
        y = self.layer3(y)
        y = self.layer4(y)
        y = self.layer5(y)
        y = self.layer6(y)
        y = self.layer7(y)
        y = self.layer8(y)
        y = self.layer9(y)
        y = self.layer10(y)
        y = self.layer11(y)
        y = self.layer12(y)
        y = self.layer13(y)
        y = self.layer14(y)
        y = self.layer15(y)
        y = self.layer16(y)
        y = self.layer17(y)
        y = self.layer18(y)
        y = self.layer19(y)
        y = self.layer20(y)
        y = self.layer21(y)
        y = self.layer22(y)
        # only include Linear layer when training
        y = self.layer23(y)        
        # return closs_out, y
        return y

In [ ]:
# ResNet
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.layer0 = nn.Sequential(nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(64), nn.ReLU(), nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
        self.layer1 = self.res_layer(block, layers[0], intermediate_channels=64, stride=1)
        self.layer2 = self.res_layer(block, layers[1], intermediate_channels=128, stride=2)
        self.layer3 = self.res_layer(block, layers[2], intermediate_channels=256, stride=2)
        self.layer4 = self.res_layer(block, layers[3], intermediate_channels=512, stride=2)
        self.layer5 = nn.Sequential(nn.Conv2d(2048, 1024, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(1024), nn.ReLU())
        self.layer6 = nn.Sequential(nn.Conv2d(1024, 512, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(512), nn.ReLU())
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * 4, num_classes, bias=False)

    def forward(self, x):
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.layer6(x)
        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)
        return x

    def res_layer(self, block, num_residual_blocks, out_channels, stride):
        downsample = None
        layers = []

        if stride != 1 or self.in_channels != out_channels * 4:
            downsample = nn.Sequential(nn.Conv2d(self.in_channels, out_channels * 4, kernel_size=1, stride=stride), nn.BatchNorm2d(out_channels * 4))
        layers.append(ResidualBlock(self.in_channels, out_channels, downsample, stride))

        self.in_channels = out_channels * 4

        for i in range(num_residual_blocks - 1):
            layers.append(ResidualBlock(self.in_channels, out_channels))

        return nn.Sequential(*layers)


class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, downsample=None):
        super(block, self).__init__()
        self.expansion = 4
        self.layer1 = nn.Sequential(nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0), nn.BatchNorm2d(out_channels), nn.ReLU())
        self.layer2 = nn.Sequential(nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(out_channels), nn.ReLU())
        self.conv3 = nn.Sequential(nn.Conv2d(out_channels, out_channels * self.expansion, kernel_size=1, stride=1, padding=0), nn.BatchNorm2d(out_channels * self.expansion))
        self.relu = nn.ReLU()
        self.downsample = downsample

    def forward(self, x):
        identity = x.clone()
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        if self.downsample is not None:
            identity = self.downsample(identity)
        x += identity
        x = self.relu(x)
        return x


def ResNet50(num_classes=4000):
    return ResNet(block, [3, 4, 6, 3], num_classes)

In [ ]:
class CenterLoss(nn.Module):
    """
    Args:
        num_classes (int): number of classes.
        feat_dim (int): feature dimension.
    """
    def __init__(self, num_classes, feat_dim):
        super(CenterLoss, self).__init__()
        self.num_classes = num_classes
        self.feat_dim = feat_dim
        
        self.centers = nn.Parameter(torch.randn(self.num_classes, self.feat_dim).cuda())

    def forward(self, x, labels):
        """
        Args:
            x: feature matrix with shape (batch_size, feat_dim).
            labels: ground truth labels with shape (batch_size).
        """
        batch_size = x.size(0)
        distmat = torch.pow(x, 2).sum(dim=1, keepdim=True).expand(batch_size, self.num_classes) + \
                  torch.pow(self.centers, 2).sum(dim=1, keepdim=True).expand(self.num_classes, batch_size).t()
        distmat.addmm_(1, -2, x, self.centers.t())

        classes = torch.arange(self.num_classes).long().cuda()
        labels = labels.unsqueeze(1).expand(batch_size, self.num_classes)
        mask = labels.eq(classes.expand(batch_size, self.num_classes))

        dist = []
        for i in range(batch_size):
            value = distmat[i][mask[i]]
            value = value.clamp(min=1e-12, max=1e+12) # for numerical stability
            dist.append(value)
        dist = torch.cat(dist)
        loss = dist.mean()

        return loss

In [ ]:
model = ResNet50().cuda()
criterion_label = nn.CrossEntropyLoss()
# criterion_closs = CenterLoss(4000, 2048)
optimizer_label = torch.optim.SGD(model.parameters(), lr=0.15, weight_decay=5e-5, momentum=0.9)
# optimizer_closs = torch.optim.SGD(criterion_closs.parameters(), lr=0.5)
# closs_weight = 0.001
scheduler = optim.lr_scheduler.StepLR(optimizer_label, step_size=1, gamma=0.85)

In [ ]:
# validation
def validate(model, batch_val):
    num_correct = 0
    for i, (batch_data, batch_labels) in enumerate(batch_val):
            batch_labels = batch_labels.cuda()
            batch_data = batch_data.cuda()
            # print(batch_labels.shape)
            outputs = model(batch_data)
            pred = torch.argmax(outputs, dim=1)
            current_correct = len([pred[i] for i in range(len(pred)) if pred[i] == batch_labels[i]])
            num_correct += current_correct
    # print(num_correct, len(batch_val))
    accuracy = num_correct / (len(batch_val) * 200)
    return accuracy

In [ ]:
# training
val_accuracies = []
test_accuracies = []
num_epochs = 5
for epoch in range(num_epochs):
    for i, (batch_data, batch_labels) in enumerate(batch_train):
        batch_labels = batch_labels.cuda()
        batch_data = batch_data.cuda()
        running_loss = 0.0
        optimizer_label.zero_grad()
        # optimizer_closs.zero_grad()
        outputs = model(batch_data)
        l_loss = criterion_label(outputs, batch_labels.long())
        loss = l_loss
        l_loss.backward()
        optimizer_label.step()
        running_loss += loss.item()
        if i % 100 == 99:
            print('[%d, %5d] loss: %.10f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0
    scheduler.step() 
    val_accu = validate(model, batch_val)
    test_accu = validate(model, batch_test)
    print(val_accu, test_accu)
    val_accuracies.append(val_accu)
    test_accuracies.append(test_accu)
print('Finished Training')

In [ ]:
from matplotlib import pyplot as plt

plt.plot(range(1, 8), val_accuracies)
plt.plot(range(1, 8), test_accuracies)
plt.plot(range(1, 8), scores)
plt.ylabel('Accuracy on validation & Test')
plt.xlabel('Epoch')
plt.show()

In [ ]:
torch.save(model.state_dict(), "model_res50_aug_e18.pt")

In [ ]:
model.load_state_dict(torch.load('model_res50_aug_e13.pt'))

<All keys matched successfully>

In [ ]:
val_accu = validate(model, batch_val)
test_accu = validate(model, batch_test)
print(val_accu, test_accu)

In [ ]:
# just a small test
first_class = [i for i in range(100) if train_loader[i][1] == 0]
twentieth_class = [i for i in range(2000) if train_loader[i][1] == 20]
similarity1 = []
similarity2 = []
embedding0, _ = model(train_loader[0][0].unsqueeze(0).cuda())
for i in first_class:
  embedding, _ = model(train_loader[i][0].unsqueeze(0).cuda())
  similarity1.append(cos_sim(embedding0, embedding).cpu().detach().numpy().item())
for i in twentieth_class:
  embedding, _ = model(train_loader[i][0].unsqueeze(0).cuda())
  similarity2.append(cos_sim(embedding0, embedding).cpu().detach().numpy().item())
print(min(similarity1), max(similarity1), len(similarity1), similarity1)
print(min(similarity2), max(similarity2), len(similarity2), similarity2)

In [ ]:
# !mkdir verification
!cp -r ./verification_data ./verification/verification_data

In [ ]:
# one way of loading verification data (that is not used)
veri_loader = torchvision.datasets.ImageFolder('verification/', transform=torchvision.transforms.ToTensor())

In [ ]:
print(len(veri_loader))
print(veri_loader[0], veri_loader[0][0].shape)

In [ ]:
def img_to_tensor(img_path):
  image = Image.open(img_path)
  image_tensor = ToTensor()(image).unsqueeze(0).cuda()
  return image_tensor

In [ ]:
# validation
with open('verification_pairs_val.txt', 'r') as f:
   val_labels = []
   val_imgs = []
   for line in f.readlines():
     line = line.strip('\n')
     fields = line.split(" ")
     val_imgs.append((fields[0], fields[1]))
     val_labels.append(int(fields[2]))
print(len(val_labels))

8805


In [ ]:
scores = []

In [ ]:
val_similarities = []
for img1, img2 in val_imgs:
  embedding1 = model(img_to_tensor(img1))
  embedding2 = model(img_to_tensor(img2))
  val_similarities.append(torch.cosine_similarity(embedding1, embedding2).cpu().detach().numpy().item())

In [ ]:
score = roc_auc_score(val_labels, val_similarities)
print(score)
scores.append(score)

In [ ]:
with open('verification_pairs_test.txt', 'r') as f:
  #  labels = []
   test_imgs = []
   for line in f.readlines():
     line = line.strip('\n')
     fields = line.split(" ")
     test_imgs.append((fields[0], fields[1]))
print(len(test_imgs))

51835


In [ ]:
# submission
with open('submission.csv', 'w') as f:
  f.write('Id,Category\n')
  for img1, img2 in test_imgs:
    f.write(img1 + " " + img2 + ",")
    embedding1 = model(img_to_tensor(img1))
    embedding2 = model(img_to_tensor(img2))
    cos = torch.cosine_similarity(embedding1, embedding2).cpu().detach().numpy().item()
    f.write(str(cos) + "\n")

In [ ]:
!kaggle competitions submit -c 11-785-fall-20-homework-2-part-2 -f submission.csv -m ""